<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Embed_2_contrast_200520_MSE_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/openai/baselines to /tmp/pip-req-build-ujiwaour
  Running command git clone -q git://github.com/openai/baselines /tmp/pip-req-build-ujiwaour
     |████████████████████████████████| 1.6MB 6.7MB/s 
  Created wheel for baselines: filename=baselines-0.1.6-cp36-none-any.whl size=220664 sha256=c3533eeaa80f2dcb1a83019bc637cc932a632cb5dc0829bc091efd9501f59926
  Stored in directory: /tmp/pip-ephem-wheel-cache-k4yi3do8/wheels/42/1c/91/28314e0cd1d2cc57cf8dd18b20c4c9a0f39ae518adc13caf24
  Created wheel for gym: filename=gym-0.15.7-cp36-none-any.whl size=1648840 sha256=8bc5698ec6c5ba3a943295b1a9b1b6c473d0b7cc287dce7ab6dbe4cdf03ddc95
  Stored in directory: /root/.cache/pip/wheels/35/60/6a/f9c27ae133abaf5a5687ed2fa8ed19627d7fac5d843a27572b
Successfully built baselines gym
ERROR: gym 0.15.7 has requirement cloudpickle~=1.2.0, but you'll have cloudpickle 1.3.0 which is incompatible.
  Found existing installation: gym 0.17.2
    Uninstalling gym-0.17.2:
      Successfully unin

In [0]:
from __future__ import print_function
import os
import pickle
import sys
#sys.path.append('/content/drive/My Drive')
sys.path.append('/content/drive/My Drive/Embed_2_Contrast')
import wandb

import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import gym
import time
import matplotlib.pyplot as plt

from custom_wrappers import custom_wrapper
from encoder import make_encoder
from earlystopping import EarlyStopping_loss
from generalfunctions import General_functions
from utils import make_dir, random_crop,center_crop_image, soft_update_params, weight_init, random_color_jitter
from torch.autograd import Variable
from datacollection import Data_collection
from models import CURL, Dynamics_model
from replay_buffer import ReplayBuffer

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [0]:
!wandb login #############

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# CURL

In [0]:
class CurlAgent(object):
    ''' CURL representation learning'''
    def __init__(
        self,
        obs_shape,
        device,
        frames, 
        encoder_lr = 1e-4,
        encoder_tau = 0.001,
        encoder_feature_dim = 50, # This is the size of the embedding used for the
        dynamics_hidden_dim = 256,
        downsample = True,
        cpc_update_freq=1,
        encoder_update_freq = 1,
        random_jitter = True,
        detach_encoder=True,
        dyn_update_freq= 2        
    ):
        self.device = device
        self.cpc_update_freq = cpc_update_freq
        self.image_size = obs_shape[-2] # Changed this to the numpy dimension
        self.frames = frames
        self.detach_encoder =  detach_encoder

        self.encoder_tau = encoder_tau
        self.epoch_step = 0
        self.encoder_update_freq = encoder_update_freq
        self.random_jitter = random_jitter
        
        self.CURL = CURL(obs_shape, encoder_feature_dim,
                         encoder_feature_dim,downsample = downsample).to(self.device)
        
        self.Model = Dynamics_model(self.CURL.encoder, encoder_feature_dim,
                                    hidden_dim=dynamics_hidden_dim).to(self.device)
        
        self.cpc_optimizer = torch.optim.Adam(
                self.CURL.parameters(), lr=encoder_lr
            )
        self.dynamics_optimizer = torch.optim.Adam(
            self.Model.parameters(), lr =encoder_lr 
        )

        self.cross_entropy_loss = nn.CrossEntropyLoss()
        self.MSE_loss = nn.MSELoss() # Nawid - Added this loss for the prediction
        self.train()
    
    def train(self, training = True):
        self.training = training
        self.CURL.train(training)
        self.Model.train(training)

    def update(self, train_dataloader,val_dataloader,early_stopper_contrastive, early_stopper_dynamics):
        #torch.cuda.empty_cache() # Releases cache so the GPU has more memory
        if early_stopper_contrastive.early_stop or early_stopper_dynamics.early_stop:
            print('early stopping-Early stopping contrastive, Early stopping dynamics :',early_stopper_contrastive.early_stop, early_stopper_dynamics.early_stop)
            return

        for step, (obs, actions, next_obs, cpc_kwargs) in enumerate(train_dataloader):
            if step % self.encoder_update_freq == 0:
                soft_update_params(
                    self.CURL.encoder, self.CURL.encoder_target,
                    self.encoder_tau
                )
            if step % self.cpc_update_freq == 0:            
                obs_anchor, obs_pos = cpc_kwargs["obs_anchor"], cpc_kwargs["obs_pos"]
                self.update_cpc(obs_anchor, obs_pos) # Nawid -  Performs the contrastive loss I believe
                    
        self.validation(val_dataloader,early_stopper_contrastive, early_stopper_dynamics)
    
    def update_cpc(self, obs_anchor, obs_pos):
        obs_anchor, obs_pos = obs_anchor.to(self.device), obs_pos.to(self.device)
        if self.random_jitter:
            obs_anchor, obs_pos = random_color_jitter(obs_anchor,batch_size = obs_anchor.shape[0],frames = self.frames), random_color_jitter(obs_pos,batch_size = obs_pos.shape[0],frames= self.frames)

        z_a = self.CURL.encode(obs_anchor) # Nawid -  Encode the anchor
        z_pos = self.CURL.encode(obs_pos, ema=True) # Nawid- Encode the positive with the momentum encoder

        logits = self.CURL.compute_logits(z_a, z_pos) #  Nawid- Compute the logits between them
        labels = torch.arange(logits.shape[0]).long().to(self.device)
        loss = self.cross_entropy_loss(logits, labels)
        wandb.log({'Contrastive Training loss':loss.item()})

        self.cpc_optimizer.zero_grad()
        loss.backward()
        self.cpc_optimizer.step()  # Nawid - Used to update the cpc
    
    def update_dynamics(self, obs,actions, next_obs):
        obs, actions, next_obs = obs.to(self.device),actions.to(self.device), next_obs.to(device)
        
        prediced_next_latent = self.Model(obs,actions,detach_encoder = self.detach_encoder) # only trains the fully connected part of the output, features from the encoder are not trained
        next_latent = self.CURL.encode(next_obs,detach=True) # no gradients will flow from this output
        prediction_loss = self.MSE_loss(prediction,labels)
        wandb.log({'Dynamics Training loss':prediction_loss.item()}) #  Need to use .item otherwise the loss will still be kept which will reduce the memory on the GPU

        self.dynamics_optimizer.zero_grad()
        prediction_loss.backward()
        self.dynamics_optimizer.step()
    


    def validation(self, dataloader,early_stopper_contrastive, early_stopper_dynamics):
        epoch_contrastive_loss = 0
        epoch_dynamics_loss = 0
        self.CURL.eval()
        self.Model.eval()
        with torch.no_grad():
            for i, (obs, actions, next_obs, cpc_kwargs) in enumerate(dataloader):
                obs_anchor, obs_pos = cpc_kwargs["obs_anchor"], cpc_kwargs["obs_pos"]
                obs, obs_anchor,obs_pos = obs.to(self.device), obs_anchor.to(self.device), obs_pos.to(self.device)
                actions, next_obs = actions.to(self.device), next_obs.to(self.device)
                if self.random_jitter:
                    obs_anchor, obs_pos =  random_color_jitter(obs_anchor,batch_size = obs_anchor.shape[0],frames = self.frames), random_color_jitter(obs_pos,batch_size = obs_pos.shape[0],frames= self.frames)

                ''' Code to check the appearance of the image
                image = obs_pos[0]
                image = image.permute(1, 2, 0)
                plt.imshow(image)
                plt.figure()
                plt.show()
                return 
                ''' 
                actions, next_obs = actions.to(self.device), next_obs.to(self.device)
                z_a = self.CURL.encode(obs_anchor) # Nawid -  Encode the anchor
                z_pos = self.CURL.encode(obs_pos, ema=True) # Nawid- Encode the positive with the momentum encoder
                logits = self.CURL.compute_logits(z_a, z_pos) #  Nawid- Compute the logits between them
                labels = torch.arange(logits.shape[0]).long().to(self.device)
                loss = self.cross_entropy_loss(logits, labels)
                epoch_contrastive_loss += loss.item()
                
                prediced_next_latent = self.Model(obs,actions,detach_encoder = self.detach_encoder) # only trains the fully connected part of the output, features from the encoder are not trained
                next_latent = self.CURL.encode(next_obs,detach=True) # no gradients will flow from this output
                prediction_loss = self.MSE_loss(prediced_next_latent,next_latent)
                epoch_dynamics_loss += prediction_loss.item()

            average_epoch_contrastive_loss = epoch_contrastive_loss/(i+1)
            average_epoch_dynamics_loss = epoch_dynamics_loss/(i+1)

            self.epoch_step += 1 # increase epoch counter
            wandb.log({'Contrastive Validation loss':average_epoch_contrastive_loss, 'Dynamics Validation loss':average_epoch_dynamics_loss,'epoch': self.epoch_step})

            print('epoch:', self.epoch_step)
            early_stopper_contrastive(average_epoch_contrastive_loss,self.CURL,self.cpc_optimizer)
            early_stopper_dynamics(average_epoch_dynamics_loss, self.Model, self.dynamics_optimizer)

        self.train()
    
def make_agent(obs_shape, device, dict_info):
    return CurlAgent(
        obs_shape = obs_shape,
        device = device,
        frames = dict_info['frames'],
        random_jitter = dict_info['random_jitter'],
        encoder_update_freq =dict_info['encoder_update_freq'],
        dyn_update_freq =dict_info['dynamics_update_freq'],
        encoder_feature_dim = dict_info['encoder_feature_dim'], #  size of the embedding from the projection head
        encoder_lr = dict_info['encoder_lr'],
        encoder_tau = dict_info['encoder_tau'],
        downsample = dict_info['downsample'],
        dynamics_hidden_dim = dict_info['dynamics_hidden_dim'],
        detach_encoder = dict_info['detach_encoder']
    )

# Hyperparameters

In [0]:
ENV_NAME = 'MsPacmanDeterministic-v4'
n_actions = 4 

data_transform = transforms.Compose([
                                    transforms.ToTensor()])

no_agents = 5
state_space = no_agents*2 
parse_dict= {'pre_transform_image_size':100,
             'image_size':84,
             'frame_stack':False,
             'frames': 1,
             'state_space':state_space,
             'train_capacity':100,#50000,
             'val_capacity':100,#20000,
             'num_train_epochs':20,
             'batch_size':512,
             'random_crop': False,
             'encoder_update_freq':1,
             'dynamics_update_freq':1,
             'encoder_feature_dim':128,
             'dynamics_hidden_dim': 256,
             'encoder_lr':1e-3,
             'encoder_tau':0.05, # value used for atari experiments in curl
             'downsample':True,
             'encoder_type':'Impala',
             'grayscale': False,
             'load_pretrain_model': False,
             'walls_present':True,
             'pretrain_model':False,
             'save_data':False,
             'num_pretrain_epochs':25,
             'transform': data_transform,
             'random_jitter':True,
             'detach_encoder':True
            }

#custom_name = 'rand_crop-' +str(parse_dict['random_crop'])  + '_gray-' + str(parse_dict['grayscale']) + '_walls-' +str(parse_dict['walls_present'])  + '_pretrain-' + str(parse_dict['pretrain_model'])
custom_name = 'Contrastive_hp_testing_random_jitter-'+str(parse_dict['random_jitter']) + '_encoder_tau-' +str(parse_dict['encoder_tau']) 
wandb.init(entity="nerdk312",name=custom_name, project="Embed2Contrast",config = parse_dict)

possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)

config = wandb.config

if parse_dict['load_pretrain_model']:
    config.pretrained_model = pretrain_model_dir

# Data collection
data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict,parse_dict['train_capacity'])
val_data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict, parse_dict['val_capacity'])

data_object.gather_random_trajectories(5000)
val_data_object.gather_random_trajectories(5000)

data_object.replay_buffer.crop_control(parse_dict['random_crop'])
val_data_object.replay_buffer.crop_control(parse_dict['random_crop'])

# dataloader
train_dataloader = DataLoader(data_object.replay_buffer, batch_size = parse_dict['batch_size'], shuffle = True)
val_dataloader = DataLoader(val_data_object.replay_buffer, batch_size = parse_dict['batch_size'], shuffle = True)



test_info = [0.001,0.005,0.01,0.05,0.1,0.5,1]
tests = len(test_info) + 1
#tests = 1 

#Training loop

for i in range(tests):
    print(i)
    if i >0:
	#parse_dict['encoder_tau'] = np.random.uniform(1e-4,1e-2)
        #parse_dict['dynamics_update_freq'] = np.random.uniform(1e-3,1e-2)

        #parse_dict['dynamics_update_freq'] = test_info[i-1]
        custom_name = 'Contrastive_hp_testing_dynamics-' +str(parse_dict['dynamics_update_freq'])
        wandb.init(entity="nerdk312",name=custom_name, project="Embed_2_Contrast_dynamics_update_rate_tests",config = parse_dict)

    agent = make_agent(
    obs_shape = data_object.obs_shape,
    device =data_object.device,
    dict_info = parse_dict
    )

    pretrain_model_name = 'Contrastive' +'_' + data_object.ts
    dynamics_model_name = 'Dynamics' +'_' + data_object.ts

    early_stopping_contrastive = EarlyStopping_loss(patience=3, verbose=True, wandb=wandb, name=pretrain_model_name)
    early_stopping_dynamics = EarlyStopping_loss(patience=3, verbose=True, wandb=wandb, name=dynamics_model_name)

    for step in range(parse_dict['num_train_epochs']):
        if early_stopping_contrastive.early_stop or early_stopping_dynamics.early_stop: #  Stops the training if early stopping counter is hit
            break
        agent.update(train_dataloader,val_dataloader,early_stopping_contrastive,early_stopping_dynamics)

    wandb.join()









'''
for i in range(tests):  
    print(i)  
    if i >0:
        #parse_dict['encoder_tau'] = np.random.uniform(1e-4,1e-2)
        #parse_dict['encoder_lr'] = np.random.uniform(1e-3,1e-2)
        parse_dict['random_jitter'] = True 
        parse_dict['encoder_tau'] = test_info[i-1]
        custom_name = 'Contrastive_hp_testing_random_jitter-'+str(parse_dict['random_jitter']) + '_encoder_tau-' +str(parse_dict['encoder_tau']) 
        wandb.init(entity="nerdk312",name=custom_name, project="Contrastive_learning",config = parse_dict)

    agent = make_agent(
    obs_shape = data_object.obs_shape,
    device =data_object.device,
    dict_info = parse_dict
    )

    pretrain_model_name = 'Contrastive' +'_' + data_object.ts
    dynamics_model_name = 'Dynamics' +'_' + data_object.ts

    early_stopping_contrastive = EarlyStopping_loss(patience=3, verbose=True, wandb=wandb, name=pretrain_model_name)
    early_stopping_dynamics = EarlyStopping_loss(patience=3, verbose=True, wandb=wandb, name=dynamics_model_name)

    for step in range(parse_dict['num_train_epochs']):
        if early_stopping_contrastive.early_stop or early_stopping_dynamics.early_stop: #  Stops the training if early stopping counter is hit
            break    
        agent.update(train_dataloader,val_dataloader,early_stopping_contrastive,early_stopping_dynamics)

    wandb.join()
'''
    

cpu
cpu
trajectory number: 0
trajectory number: 0
0
epoch: 1
Validation loss decreased/improved for Contrastive_20-05_14:21  (100000000000.000000 --> 6.780341).  Saving model ...
Validation loss decreased/improved for Dynamics_20-05_14:21  (100000000000.000000 --> 7.064348).  Saving model ...
epoch: 2
EarlyStopping for Contrastive_20-05_14:21 counter: 1 out of 3
Validation loss decreased/improved for Dynamics_20-05_14:21  (7.064348 --> 6.788999).  Saving model ...
epoch: 3
EarlyStopping for Contrastive_20-05_14:21 counter: 2 out of 3
EarlyStopping for Dynamics_20-05_14:21 counter: 1 out of 3
epoch: 4
EarlyStopping for Contrastive_20-05_14:21 counter: 3 out of 3
Contrastive_20-05_14:21 has stopped
EarlyStopping for Dynamics_20-05_14:21 counter: 2 out of 3
1


epoch: 1
Validation loss decreased/improved for Contrastive_20-05_14:21  (100000000000.000000 --> 28.308090).  Saving model ...
Validation loss decreased/improved for Dynamics_20-05_14:21  (100000000000.000000 --> 4.997063).  Saving model ...
epoch: 2
Validation loss decreased/improved for Contrastive_20-05_14:21  (28.308090 --> 13.767324).  Saving model ...
EarlyStopping for Dynamics_20-05_14:21 counter: 1 out of 3
epoch: 3
Validation loss decreased/improved for Contrastive_20-05_14:21  (13.767324 --> 8.107104).  Saving model ...
EarlyStopping for Dynamics_20-05_14:21 counter: 2 out of 3


KeyboardInterrupt: ignored

In [0]:
agent = make_agent(
            obs_shape = data_object.obs_shape,
            device =data_object.device,
            dict_info = parse_dict
        )

agent.Model

Dynamics_model(
  (encoder): Encoder(
    (layer1): Sequential(
      (0): Conv2dSame(
        (net): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
        )
      )
      (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ReLU()
      (3): ResidualBlock(
        (block): Sequential(
          (0): Conv2dSame(
            (net): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
            )
          )
          (1): ReLU()
          (2): Conv2dSame(
            (net): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
            )
          )
        )
      )
      (4): ReLU()
      (5): ResidualBlock(
        (block): Sequential(
          (0): Conv2dSame(
            (net): Sequential(
              (0): Reflec

In [0]:
test_info = (0.005), (0.01), (0.08), (0.01)
#test_info = (10),(20),(50),(100),(200),(500)
tests = len(test_info) + 1
test_info[0]
#parse_dict['dynamics_update_freq'] = test_info[0][0]


# Old code

Data - Collection

In [0]:
initial_time = time.time()
data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict,parse_dict['train_capacity_1'])
if parse_dict['load_trajectories']:
    data_object.replay_buffer.load(preloaded_train_data_1)
else:
    data_object.gather_random_trajectories(5000)

final_time = time.time() - initial_time

cpu
trajectory number: 0
trajectory number: 10
trajectory number: 20
trajectory number: 30
trajectory number: 40
trajectory number: 50
trajectory number: 60
trajectory number: 70
trajectory number: 80
trajectory number: 90
trajectory number: 100
trajectory number: 110
trajectory number: 120
trajectory number: 130
trajectory number: 140
trajectory number: 150
trajectory number: 160
trajectory number: 170
trajectory number: 180
trajectory number: 190
trajectory number: 200
trajectory number: 210
trajectory number: 220
trajectory number: 230
trajectory number: 240
trajectory number: 250
trajectory number: 260
trajectory number: 270
trajectory number: 280
trajectory number: 290
trajectory number: 300
trajectory number: 310
trajectory number: 320
trajectory number: 330
trajectory number: 340
trajectory number: 350
trajectory number: 360
trajectory number: 370
trajectory number: 380
trajectory number: 390
trajectory number: 400
trajectory number: 410
trajectory number: 420
trajectory number:

In [0]:
data_object.replay_buffer.obses.shape

(100000, 100, 100, 12)

In [0]:
data_object_2 = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict,parse_dict['train_capacity_2'])
if parse_dict['load_trajectories']:
    data_object_2.replay_buffer.load(preloaded_train_data_2)
else:
    data_object_2.gather_random_trajectories(5000)

data_object.replay_buffer.obses = np.concatenate((data_object.replay_buffer.obses, data_object_2.replay_buffer.obses), axis=0)
data_object.replay_buffer.actions = np.concatenate((data_object.replay_buffer.actions, data_object_2.replay_buffer.actions), axis=0)
data_object.replay_buffer.state_changes = np.concatenate((data_object.replay_buffer.state_changes, data_object_2.replay_buffer.state_changes), axis=0)
del data_object_2

cuda
trajectory number: 0
trajectory number: 10


In [0]:
data_object_3 = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict,parse_dict['train_capacity_3'])
if parse_dict['load_trajectories']:
    data_object_3.replay_buffer.load(preloaded_train_data_2)
else:
    data_object_3.gather_random_trajectories(5000)

data_object.replay_buffer.obses = np.concatenate((data_object.replay_buffer.obses, data_object_3.replay_buffer.obses), axis=0)
data_object.replay_buffer.actions = np.concatenate((data_object.replay_buffer.actions, data_object_3.replay_buffer.actions), axis=0)
data_object.replay_buffer.state_changes = np.concatenate((data_object.replay_buffer.state_changes, data_object_3.replay_buffer.state_changes), axis=0)

del data_object_3

cuda
trajectory number: 0


In [0]:
val_data_object = Data_collection(ENV_NAME,n_actions,possible_positions, parse_dict, parse_dict['val_capacity'])
if parse_dict['load_trajectories']:
    val_data_object.replay_buffer.load(preloaded_val_data)
else:
    val_data_object.gather_random_trajectories(5000)

train_dataloader = DataLoader(data_object.replay_buffer, batch_size = 256, shuffle = True)
val_dataloader = DataLoader(val_data_object.replay_buffer, batch_size = 256, shuffle = True)

cpu
trajectory number: 0
trajectory number: 10
trajectory number: 20
trajectory number: 30
trajectory number: 40
trajectory number: 50
trajectory number: 60
trajectory number: 70
trajectory number: 80
trajectory number: 90
trajectory number: 100
trajectory number: 110
trajectory number: 120
trajectory number: 130
trajectory number: 140
trajectory number: 150
trajectory number: 160
trajectory number: 170
trajectory number: 180
trajectory number: 190


In [0]:
agent = make_agent(
        obs_shape = data_object.obs_shape,
        device =data_object.device,
        dict_info = parse_dict
    )

dyn_model_name = 'Dynamics'+ '_' + data_object.ts
pretrain_model_name = 'Contrastive' +'_' + data_object.ts

early_stopping_dynamics = EarlyStopping_loss(patience=3, verbose=True, wandb=wandb, name=dyn_model_name)
early_stopping_contrastive = EarlyStopping_loss(patience=3, verbose=True, wandb=wandb, name=pretrain_model_name)
env = gym.make(ENV_NAME)
env = custom_wrapper(env, grayscale = parse_dict['grayscale'])
obs = env.reset()
info_labels = env.labels()
state = data_object.state_conversion(info_labels)


if parse_dict['pretrain_model']:
    for pretrain_step in range(num_pretrain_steps):
        if early_stopping_contrastive.early_stop: #  Stops the training if early stopping counter is hit
            break
        agent.pretrain(train_dataloader,val_dataloader, early_stopping_contrastive)


for step in range(parse_dict['num_train_steps']):
    if early_stopping_dynamics.early_stop: #  Stops the training if early stopping counter is hit
        break    
    agent.update(train_dataloader,val_dataloader,early_stopping_dynamics)

epoch: 1
val prediction: tensor([[-7.5782e-01,  6.3639e-01,  2.7453e+00,  1.6735e-01],
        [ 2.1720e-01, -6.0008e-01,  2.7503e+00,  7.5085e-02],
        [ 8.9513e-02,  1.0737e+00,  2.8843e+00,  2.1094e-01],
        [-9.6177e-01,  2.3771e+00,  2.9850e+00,  2.9699e-01],
        [-4.8269e-02,  4.9470e-01,  2.8341e+00, -3.1386e-02],
        [ 8.3307e-02,  3.8426e-03, -2.4567e+00,  2.5947e-01],
        [ 7.1103e-01,  1.2564e+00,  1.7838e-01, -3.4542e+00],
        [-8.3351e-01,  1.6868e+00,  2.9461e+00,  2.2221e-01],
        [-1.2171e+00,  1.2245e-01, -2.6939e+00,  1.5920e-01],
        [ 2.3942e-01,  7.3075e-01,  2.3275e-01,  4.2219e+00],
        [ 4.6087e-01,  1.5841e+00, -2.4413e+00,  2.6704e-01],
        [ 2.3502e-01,  1.6857e+00,  2.2717e+00,  6.4601e-04],
        [ 2.2103e-01,  2.3592e+00,  2.5964e+00, -7.9499e-02],
        [ 2.8328e-01, -2.2833e-01, -2.9107e+00, -7.1508e-02],
        [-4.4191e-01,  1.6260e+00,  2.7798e+00,  4.2085e-01],
        [-6.1588e-02,  2.2736e+00, -2.5601e+0

KeyboardInterrupt: ignored